In [17]:
import pathlib

import tensorflow as tf
import numpy as np

import cv2

from PIL import Image
import matplotlib.pyplot as plt
import os
import pathlib

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import imagenet_utils

from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.inception_v3 import *


from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

2.1.0


## Montando uma Rede Neural Convolucional (CNN)
* Convolução
    * conv2d
* Pooling
    * MaxPooling2D    
    * AveragePooling2D

## Avaliando a Rede Neural Com Dados da Base Mnist (Dígitos
* Dividindo os valores por 255 para normalização (transforma para 0~1)

In [18]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Normalmente, redes CNNs são montadas da seguinte forma:
* Sequência de camadas de convolução e pooling
* Camada flatten
    * Realiza uma operação de transformar um vetor com múltiplas dimensões para um vetor com apenas uma dimensão. Necessário para adequar corretamente o resultado da convolução com a camada de classificação
* Camada densa com ativação softmax para classificação

In [50]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))

* Após o processo de Convolução é utilizada uma camada Dense para classificar entre as classes

In [51]:
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

In [52]:
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model.fit(train_images, train_labels, epochs=5)#, batch_size=64)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 144us/sample - loss: 0.2132 - accuracy: 0.9390
Epoch 2/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.0859 - accuracy: 0.9757
Epoch 3/5
60000/60000 [==============================] - 8s 139us/sample - loss: 0.0692 - accuracy: 0.9795
Epoch 4/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.0603 - accuracy: 0.9829
Epoch 5/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.0559 - accuracy: 0.9845


In [55]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 103us/sample - loss: 0.0579 - accuracy: 0.9811


In [27]:
test_acc

0.9821

## Utilizando rede Neural Treinada para realizar Predição

In [87]:
model = VGG16()

In [88]:
image = load_img('figs/mug.jpg', target_size=(224, 224))
#image = load_img('figs/mug.jpg', target_size=(299, 299))

In [89]:
# Converter pixels em um array
image = img_to_array(image)

# acertar formato do array para o modelo
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# preprocessamento da imagem
image = preprocess_input(image)

# calcular probabilidade considerando todos os "labels"
yhat = model.predict(image)

# Codificar as probabilidades retornadas nos "labels"
label = decode_predictions(yhat)

for (i, (imagenetID, label, prob)) in enumerate(label[0]):
	print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

1. toilet_tissue: 10.89%
2. mosquito_net: 7.59%
3. shower_curtain: 5.20%
4. toilet_seat: 3.65%
5. paper_towel: 1.75%


In [90]:
!ls figs/mug.jpg

figs/mug.jpg


In [93]:
model = InceptionV3(weights='imagenet')

img = image.load_img('figs/mug.jpg', target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

y = model.predict(x)
for index, res in enumerate(decode_predictions(y)[0]):
    print('{}. {}: {:.3f}%'.format(index + 1, res[1], 100 * res[2]))

1. coffee_mug: 66.749%
2. cup: 12.973%
3. teapot: 0.715%
4. coffeepot: 0.699%
5. pitcher: 0.490%


## Comparando MLP x CNN

In [6]:
main_dir="/home/silvio/git/datasets/dogsCatsDB/"
train_dir = "train"
path = os.path.join(main_dir,train_dir)

X = []
y = []
convert = lambda category : int(category == 'dog')

# Percorre os arquivos no diretório de imagens para treinamento

def create_test_data(path):
    for p in os.listdir(path):

        # categoria da imagem é definida pelo nome do arquivo
        category = p.split(".")[0]

        # define categoria como 0 ou 1
        category = convert(category)
        
        # Abre a imagem usando opencv em escala de cinza
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)

        # Redimensionamento para 80 x 80 pixels
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        
        X.append(new_img_array)
        y.append(category)

path="/home/silvio/git/datasets/dogsCatsDB/train"

create_test_data(path)

print(np.array(X).shape)
print(np.array(y).shape)
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

print(X)
print(y)


(21000, 80, 80)
(21000,)
[[[[241]
   [236]
   [235]
   ...
   [252]
   [252]
   [252]]

  [[245]
   [240]
   [242]
   ...
   [252]
   [252]
   [252]]

  [[253]
   [250]
   [250]
   ...
   [252]
   [252]
   [252]]

  ...

  [[167]
   [166]
   [177]
   ...
   [252]
   [252]
   [252]]

  [[149]
   [154]
   [142]
   ...
   [252]
   [252]
   [252]]

  [[111]
   [139]
   [120]
   ...
   [252]
   [252]
   [252]]]


 [[[152]
   [ 66]
   [109]
   ...
   [232]
   [206]
   [133]]

  [[102]
   [146]
   [ 96]
   ...
   [159]
   [138]
   [202]]

  [[114]
   [ 91]
   [ 66]
   ...
   [175]
   [122]
   [120]]

  ...

  [[163]
   [173]
   [195]
   ...
   [150]
   [156]
   [224]]

  [[181]
   [165]
   [171]
   ...
   [135]
   [225]
   [170]]

  [[223]
   [155]
   [148]
   ...
   [163]
   [166]
   [191]]]


 [[[172]
   [156]
   [153]
   ...
   [128]
   [107]
   [ 96]]

  [[176]
   [162]
   [159]
   ...
   [108]
   [103]
   [108]]

  [[165]
   [167]
   [156]
   ...
   [ 72]
   [ 57]
   [ 84]]

  ...

  [[1

In [103]:
print(X.shape)
print(y.shape)

(21000, 80, 80, 1)
(21000,)


# CNN
* conv2d
* maxpooling2d
* padding
    * same = aplica valores 0 ao redor de toda imagem, mantendo tamanho original da matriz após aplicar filtros   

In [16]:
y_binary = to_categorical(y)
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 1), padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, y_binary, epochs=5)#, batch_size=64)

test_loss, test_acc = model.evaluate(X, y_binary)

print(test_acc)

Train on 21000 samples
Epoch 1/5
21000/21000 [==============================] - 5s 257us/sample - loss: 18.0992 - accuracy: 0.5786
Epoch 2/5
21000/21000 [==============================] - 3s 167us/sample - loss: 0.6419 - accuracy: 0.6414
Epoch 3/5
21000/21000 [==============================] - 3s 165us/sample - loss: 0.5901 - accuracy: 0.6840
Epoch 4/5
21000/21000 [==============================] - 4s 168us/sample - loss: 0.5527 - accuracy: 0.7105
Epoch 5/5
21000/21000 [==============================] - 2s 110us/sample - loss: 0.4580 - accuracy: 0.7657
0.7657143


In [ ]:
# MLP

In [110]:
Xmlp = np.array(X).reshape(21000,6400)

In [112]:
y_binary = to_categorical(y)
model = models.Sequential()

model.add(layers.Dense(128, activation='relu', input_shape=(6400,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(Xmlp, y_binary, epochs=5)

test_loss, test_acc = model.evaluate(Xmlp, y_binary)

print(test_acc)

Train on 21000 samples
Epoch 1/5
21000/21000 [==============================] - 3s 162us/sample - loss: 29.1505 - accuracy: 0.5045
Epoch 2/5
21000/21000 [==============================] - 3s 152us/sample - loss: 1.1081 - accuracy: 0.5009
Epoch 3/5
21000/21000 [==============================] - 3s 152us/sample - loss: 1.0036 - accuracy: 0.4969
Epoch 4/5
21000/21000 [==============================] - 3s 151us/sample - loss: 1.0490 - accuracy: 0.4949
Epoch 5/5
21000/21000 [==============================] - 2s 95us/sample - loss: 0.6931 - accuracy: 0.5001
0.50009525
